In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np
import random
import string
import sys
import csv
import json
from lxml import objectify
from io import StringIO

In [31]:
df1 = DataFrame(np.arange(1,13).reshape(3,4), columns = list('abcd'))
df1['message'] = ['hello','world','foo']

In [32]:
df1.to_csv('examples\\ex1.csv',index = False)

In [64]:
df1 = pd.read_csv('examples\\ex1.csv',index_col = ['message'],skiprows = [2])
df1

,a,b,c,d
message,,,,
hello,1,2,3,4
foo,9,10,11,12


In [48]:
df2 = DataFrame(np.arange(1,17).reshape(8,2),columns = ['value1','value2']) 
df2['key1'] = ['one'] * 4 + ['two']*4
df2['key2'] = list('abcdefgh')

In [49]:
df2.to_csv('examples\\ex2.csv',index = False)

In [50]:
df2 = pd.read_csv('examples\\ex2.csv',index_col = ['key1','key2'])
df2

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  e          9      10
     f         11      12
     g         13      14
     h         15      16

In [51]:
df2.loc['one']

,value1,value2
key2,,
a,1,2
b,3,4
c,5,6
d,7,8


In [58]:
pd.read_table('examples\\ex3.txt',sep = '\s',engine = 'python')

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [84]:
df3 = pd.read_csv('examples\\ex5.csv')
df3

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [75]:
df4 = pd.read_csv('examples\\ex5.csv')
df4 == df3 # nan 和任何进行比较，结果都是False，见下： 
# np.nan == np.nan 返回仍然是False

,something,a,b,c,d,message
0,True,True,True,True,True,False
1,True,True,True,False,True,True
2,True,True,True,True,True,True


In [67]:
df3.isnull()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [80]:
sentinels = {'message':['foo','NA'],'something':['one']}
df5 = pd.read_csv('examples\\ex5.csv',na_values = sentinels)
df5 # 在na_values的参数中有一些默认被当作空值的字符串，例如NA，N/A

,something,a,b,c,d,message
0,NaN,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


In [3]:
pd.read_csv('examples\\ex6.csv',nrows = 5) # 没有这个nrows的话，将全部读进来

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [25]:
chunker = pd.read_csv('examples\\ex6.csv',chunksize = 1000) 
# chunksize将文件按照chunksize进行划分，读取了所有的文件（默认）,生成了一个(类似于)迭代器

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value = 0)
    
tot = tot.sort_values(ascending = False)

In [83]:
tot.rename('key',inplace = True)

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
V    328.0
I    327.0
U    326.0
P    324.0
D    320.0
A    320.0
R    318.0
Y    314.0
G    308.0
S    308.0
N    306.0
W    305.0
T    304.0
B    302.0
Z    288.0
C    286.0
4    171.0
6    166.0
7    164.0
8    162.0
3    162.0
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Name: key, dtype: float64

In [29]:
tot2 = pd.read_csv('examples\\ex6.csv')['key'].value_counts().sort_values(ascending=False) 

In [69]:
tot2 == tot.astype('float')

ValueError: Can only compare identically-labeled Series objects

In [64]:
len(tot) == len(tot2) # 为什么这里比较他们时回报错？因为他们的列名字不一样，然而用下面的方法，它又是正确的

True

In [70]:
df14 = DataFrame({1:tot,2:tot2})
df14[3] = df14[1] - df14[2]

In [82]:
(df14[3] == 0).all()

True

In [ ]:
# %load examples/out.csv
,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo

In [90]:
import sys
df3.to_csv(sys.stdout,sep = '|') # sys.out 直接在console中打印出来 重定向到控制台

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [92]:
df3.to_csv(sys.stdout,na_rep = 'null') # na_rep空值替换

,something,a,b,c,d,message
0,one,1,2,3.0,4,null
1,two,5,6,null,8,world
2,three,9,10,11.0,12,foo


In [95]:
s1 = pd.date_range('1/1/2020',periods = 7)
s2 = Series(np.arange(7),index = s1)
s2.to_csv(sys.stdout,sep = ',')

2020-01-01,0
2020-01-02,1
2020-01-03,2
2020-01-04,3
2020-01-05,4
2020-01-06,5
2020-01-07,6


 # working with delimited formats

In [ ]:
# %load examples\ex7.csv
"a","b","c"
"1","2","3"
"1","2","3"


In [98]:
f = open('examples\\ex7.csv','r')

In [100]:
reader  = csv.reader(f)

In [101]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [106]:
with open('examples\\ex7.csv','r',encoding = 'utf-8') as f:
    reader = list(csv.reader(f))

col = reader[0]
values = reader[1:]
ex7_dict = {h:v for h,v in zip(col,zip(*values))}
ex7_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [109]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
   
f = open('examples\\ex7.csv','r')
reader = csv.reader(f,dialect = my_dialect)
reader

In [110]:
with open('examples\\mydata.csv','w',encoding = 'utf-8') as f:
    writer = csv.writer(f,dialect = my_dialect)
    writer.writerow(('a','b','c'))
    writer.writerow((1,2,3))

In [ ]:
# %load examples\mydata.csv
a,b,c
1,2,3

In [112]:
with open('examples\\mydata.csv','w',encoding = 'utf-8') as f:
    f.write('a,b,c\n')
    f.write('1,2,3') 

# json data

In [133]:
 obj1 = '''{"name":"wes","places_lived":["USA","Spain","German"],"pet":null,"siblings":[{"name":"Scott","age":30},{"name":"katie","age":30}]}'''

In [135]:
obj1

'{"name":"wes","places_lived":["USA","Spain","German"],"pet":null,"siblings":[{"name":"Scott","age":30},{"name":"katie","age":30}]}'

In [136]:
result = json.loads(obj1)

In [137]:
result

{'name': 'wes',
 'pet': None,
 'places_lived': ['USA', 'Spain', 'German'],
 'siblings': [{'age': 30, 'name': 'Scott'}, {'age': 30, 'name': 'katie'}]}

In [138]:
obj2 = json.dumps(result)

In [140]:
siblings = DataFrame(result['siblings'])
siblings

,age,name
0,30,Scott
1,30,katie


In [ ]:
# %load examples\example.json
[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [145]:
df15 = pd.read_json('examples\\example.json')
df15

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [147]:
df15.to_json(sys.stdout,orient = 'records')

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

# XML and HTML:Web Scraping

In [150]:
tables = pd.read_html('examples\\fdic_failed_bank_list.html') # 返回的是一个(巨大的)list

In [156]:
type(tables)

list

In [153]:
failures = tables[0] 

In [157]:
type(failures)

pandas.core.frame.DataFrame

In [159]:
pd.to_datetime(failures['Closing Date']).dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

In [164]:
path = 'examples\\mta_perf\\Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()
root

FileNotFoundError: [Errno 2] No such file or directory: 'examples\\mta_perf\\Performance_MNR.xml'

In [4]:
atag = '<a href="www.baidu.com">baidu</a>'
root = objectify.parse(StringIO(atag)).getroot()

In [7]:
root.get('href')

'www.baidu.com'

# binary data formats

In [8]:
df20 = pd.read_csv('examples\\ex1.csv')

In [9]:
df20.to_pickle('examples\\pickle_format')

In [10]:
df21 = pd.read_pickle('examples\\pickle_format')
df21

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


# HDF

In [12]:
df22 = DataFrame({'a':np.arange(1000)})
store = pd.HDFStore('examples\\mydata.h5')

In [14]:
store['obj'] = df22
store['obj_col'] = df22.a
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples\mydata.h5
/obj                frame        (shape->[1000,1])
/obj_col            series       (shape->[1000])  

In [16]:
store.put('obj2',df22,format = 'table')

In [18]:
store.select('obj2',where = ['index >= 10 and index <= 15'])

,a
10,10
11,11
12,12
13,13
14,14
15,15


In [22]:
store.close()

In [23]:
df22.to_hdf('examples\\mydata.h5','obj3',format = 'table')

In [29]:
pd.read_hdf('examples\\mydata.h5','obj2',where = ['index < 5'])

,a
0,0
1,1
2,2
3,3
4,4


In [31]:
store1 = pd.HDFStore('examples\\mydata.h5')

In [32]:
store1

<class 'pandas.io.pytables.HDFStore'>
File path: examples\mydata.h5
/obj                frame        (shape->[1000,1])                                       
/obj2               frame_table  (typ->appendable,nrows->1000,ncols->1,indexers->[index])
/obj3               frame_table  (typ->appendable,nrows->1000,ncols->1,indexers->[index])
/obj_col            series       (shape->[1000])                                         

# reading excel files

In [3]:
df23 = pd.read_excel('examples\\ex1.xlsx','Sheet1')
df23

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
df23.to_excel('examples\\ex2.xlsx',"保存的文件")